## Imports

In [70]:
import pandas as pd
import numpy as np

### Import departures data

The source of the data is World Bank Open Data: https://data.worldbank.org/indicator/ST.INT.DPRT.  
Departures refer to residents leaving the country for tourism or business purposes.

In [71]:
departures = pd.read_csv('data/Tourism_Departures.csv', sep=';')

In [72]:
departures.head()
# Country = Country of origin of the person traveling abroad
# Year = Year in which the person traveled abroad for tourism or business purposes
# Number_of_Tourists = Number of people from the country who traveled abroad during the year.
# If someone leaves the country twice in the same year for tourism purposes, this individual will be counted twice.

,Country,Visitor_Type,Year,Value
0,ALBANIA,Overnights visitors (tourists),1995,NaN
1,ALBANIA,Same-day visitors (excursionists),1995,NaN
2,ALGERIA,Same-day visitors (excursionists),1995,NaN
3,ALGERIA,Same-day visitors (excursionists),1995,NaN
4,ALGERIA,Same-day visitors (excursionists),1995,NaN


### Import expenditures data

The source of the data is World Bank Open Data: https://data.worldbank.org/indicator/ST.INT.TRNX.CD.  
Expenditures refer to the total spending made by residents of the mentioned country when visiting foreign countries for tourism purposes. It includes:  

In [73]:
expenditures = pd.read_csv('data/Tourism_Expenditures.csv', sep=';')

In [74]:
expenditures.head()
# Country = Country of origin of the person traveling abroad
# Year = Year in which the person traveled abroad for tourism or business purposes
# Category = Passenger transport (Airfare, Items purchased on board) and Travel (Accomodation, Food, Duty-free purchases)

,Country,Category,Year,Value
0,AFGHANISTAN,NaN,1995,NaN
1,AFGHANISTAN,Travel,1995,NaN
2,AFGHANISTAN,Passenger transport,1995,NaN
3,ALBANIA,NaN,1995,NaN
4,ALBANIA,NaN,1995,19.0


## Process data

### Remove NaN values

In [75]:
# Remove NaN in departures dataframe
departures.dropna(subset=['Value'], inplace=True)

# Remove NaN in expenditures dataframe
expenditures.dropna(subset=['Value'], inplace=True)

### Remove rows corresponding to a total of Overnights visitors (tourists) and Same-day visitors (excursionists)

#### Create a function to check if a row corresponds to a sum of other rows (i.e. for a same country and year)

In [76]:
import pandas as pd
import numpy as np

def mark_total_rows(df):
    df = df.copy()
    df["Sum"] = np.nan  # Sum column is created 

    for (country, year), group in df.groupby(["Country", "Year"]):
        if len(group) == 3:
            values = group["Value"].dropna().values
            if len(values) == 3:
                a, b, c = sorted(values)
                if np.isclose(c, a + b):
                    # Trouver la ligne du total et marquer "Yes"
                    total_index = group[group["Value"] == c].index
                    if len(total_index) == 1:
                        df.loc[total_index[0], "Sum"] = "Yes"
    return df

In [77]:
# Apply the function on departures
departures = mark_total_rows(departures)

# Apply the function on expenditures
expenditures = mark_total_rows(expenditures)

C:\Users\stanislas.dhuart_art\AppData\Local\Temp\ipykernel_27712\845181461.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Yes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[total_index[0], "Sum"] = "Yes"
C:\Users\stanislas.dhuart_art\AppData\Local\Temp\ipykernel_27712\845181461.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Yes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[total_index[0], "Sum"] = "Yes"


#### Remove the rows containing Yes

In [78]:
# Remove these rows on departures
departures = departures.query('Sum != "Yes"')

# Remove these rows on expenditures
expenditures = expenditures.query('Sum != "Yes"')

### Remove the column Sum

In [79]:
# Remove the column on departures
departures.drop(columns=['Sum'], inplace=True)

# Remove the column on expenditures
expenditures.drop(columns=['Sum'], inplace=True)

### Rename the column Value

In [80]:
# Apply on departures
departures = departures.rename(columns={'Value':'Number_of_Tourists'})

# Apply on expenditures
expenditures = expenditures.rename(columns={'Value':'Expenditure'})

### Group all the categories of tourists in departures (we don't need to split between Overnight and same day visitors)

In [81]:
departures = departures.groupby(['Country','Year'], as_index=False)['Number_of_Tourists'].sum()

### Remove Passenger transport Expenditure in expenditures as we are not interested in how much tourists spend in transportation

In [82]:
expenditures = expenditures.query('Category!="Passenger transport"')

### Remove the Category column in expenditures as we don't need it anymore

In [83]:
expenditures.drop(columns='Category',inplace=True)

## Merge data

In [84]:
# Merge on rows corresponding exactly to the same country and the same year. Take only the rows included in both dataframes
df = pd.merge(departures, expenditures, on=['Country','Year'], how='inner')

## Add a column Expenditure per visitor

### Convert the Expenditure data expressed in millions of $ to number

In [85]:
df['Expenditure'] = df['Expenditure'] * 1000000

### Convert the Visitors data expressed in thousands of people to number

In [86]:
df['Number_of_Tourists'] = df['Number_of_Tourists'] * 1000

### Remove duplicates

In [87]:
df.drop_duplicates(inplace=True)

### Keep only the rows where the number of tourists exceeds 500,000

As the final objective of this project is to target visitors by their country of origin, we will only retain countries that generate at least 500,000 visitors in a given year.  
This ensures a sufficient pool of visitors should we decide to deploy a targeted campaign for a specific country.

In [88]:
df = df.query('Number_of_Tourists>500000')

### Calculate Expenditure per Visitor

In [89]:
df['Expenditure_per_Visitor'] = df['Expenditure'] / df['Number_of_Tourists']

In [90]:
df.head()

,Country,Year,Number_of_Tourists,Expenditure,Expenditure_per_Visitor
0,ALGERIA,1995,1090000.0,186000000.0,170.642202
1,ALGERIA,1996,874000.0,188000000.0,215.102975
2,ALGERIA,1997,827000.0,144000000.0,174.123337
3,ALGERIA,1998,879000.0,269000000.0,306.029579
4,ALGERIA,1999,903000.0,251000000.0,277.962348


## Keep only three columns: Country, Year and Expenditure_per_Visitor

In [91]:
df = df[['Country', 'Year', 'Expenditure_per_Visitor']]

## Export the final dataframe for data vizualisation

In [92]:
df.to_csv('Visitors_Spendings.csv')